# Sounds from Swarm

Currently generating sound from MAG_LR (F)

Sound processing based on work by Nikolai Linden-Vørnle: https://gitlab.gbar.dtu.dk/s183730/sonification-ESA-Swarm/

References:

- https://resampy.readthedocs.io/
- https://pyrubberband.readthedocs.io/
- https://panel.holoviz.org/reference/index.html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataclasses import dataclass
from time import sleep
import datetime as dt
import numpy as np
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd

from hapiclient import hapi, hapitime2datetime
import resampy
import pyrubberband as pyrb

import panel as pn

pn.extension()

In [ ]:
import tfalib_temp

## Data access through VirES+HAPI

In [ ]:
def fetch_data(t0, t1) -> pd.DataFrame:
    """Fetch data from VirES HAPI
    """
    if isinstance(t0, dt.datetime):
        t0 = t0.isoformat()
        t1 = t1.isoformat()
    data, meta = hapi(
        "https://vires.services/hapi/",
        "SW_OPER_MAGA_LR_1B",
        "Latitude,Longitude,Radius,F",
        t0,
        t1,
    )
    # Convert to dataframe
    #  To fix: this will not work with vector, e.g. B_NEC
    df = pd.DataFrame(columns=data.dtype.names, data=data)
    df = df.set_index("Timestamp")
    df.index = hapitime2datetime(df.index.values.astype(str))
    df.index = df.index.tz_convert("UTC").tz_convert(None)
    return df


def fetch_data_padded(t0, t1):
    # padding is required for the TFA toolbox
    t0 = t0 - dt.timedelta(hours=3)
    t1 = t1 + dt.timedelta(hours=3)
    return fetch_data(t0, t1)


t0, t1 = dt.datetime(2020, 3, 14), dt.datetime(2020, 3, 14, 1)
df = fetch_data_padded(t0, t1)

## Sound processing tools

In [ ]:
def normalise(x):
    """Normalise signal to +1/-1 range"""
    return (x - np.average(x)) / (np.ptp(x))


def highpass_filter(x, filter_order=8, cutoff_freq=0.0001, fs=1):
    """Apply butterworth highpass filter to remove DC offset"""
    sos = signal.butter(filter_order, cutoff_freq, 'highpass', fs=fs, output='sos')
    # w, h = signal.sosfreqz(sos)
    return signal.sosfilt(sos, x)


def resample(x, fs=44100, resampling_factor=7):
    """Resample to new frequency"""
    sr_new = int(fs / resampling_factor)
    return resampy.resample(x, fs, sr_new)


def smooth_edges(x, fraction_fade=0.1, t_fade=0.1, fs=44100):
    """Smoothing window to avoid clicks and pops at the start and end of the signal

    If fraction_fade provided, use that to apply to a fraction of the signal,
    otherwise, set fraction_fade=None to use t_fade instead
    """
    window = np.ones(len(x))
    if fraction_fade:
        length_fade = int(len(x)*fraction_fade)
    else:
        length_fade = int(t_fade * fs)
    # ramp up the fade from 0 to 1 over the duration of length_fade
    fade = np.linspace(0, 1, length_fade)
    window[:length_fade] = fade
    window[-length_fade:] = fade[::-1]
    return x * window


def time_stretch(x, fs=44100, target_length=10):
    """Stretch duration to target_length (seconds)"""
    input_length = len(x) / fs
    ts_ratio = input_length / target_length
    return pyrb.time_stretch(x, fs, ts_ratio)


def pitch_shift(x, fs=44100, octaves=1):
    """Shift the pitch by a given number of octaves"""
    return pyrb.pitch_shift(x, fs, 12*octaves)


def sound_format(x):
    """Convert to int16 for audio output"""
    return np.int16(x*32767)


def get_sound_pane(x, fs=44100):
    """Generate panel Audio pane"""
    return pn.pane.Audio(x, sample_rate=fs, sizing_mode="stretch_width")

In [ ]:
def plot_spectrogram(x, fs=1, resampling_factor=1, ymax=0.00125, figsize=(5, 5)):
    nperseg = 2**(16 + (1 - (int(0.5 + resampling_factor/2))))
    if nperseg > len(x):
        nperseg = 10000
    f, t, Sxx = signal.spectrogram(x, fs, mode='magnitude', nperseg=nperseg)
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.pcolormesh(t, f, Sxx[:], shading='gouraud', cmap='hot')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    ax.set_ylim(0, ymax)
    plt.close()
    return fig

### Now apply a pipeline created from the functions above...

In [ ]:
def _apply_audio_pipeline(x, target_length=10, shift_octaves=1):
    x = normalise(x)
    x = highpass_filter(x, filter_order=8, cutoff_freq=0.0001, fs=1)
    x = resample(x, fs=44100, resampling_factor=7)
    x = smooth_edges(x, fraction_fade=0.1)  # fs=44100, t_fade=0.1)
    x = time_stretch(x, fs=44100, target_length=target_length)
    x = pitch_shift(x, fs=44100, octaves=shift_octaves)
    # plot_spectrogram(x, fs=44100, resampling_factor=7, ymax=800)
    return x

In [ ]:
# plot_spectrogram(df["F"].values)

In [ ]:
premade_audio_data = _apply_audio_pipeline(df["F"].values, target_length=10, shift_octaves=1)
# plot_spectrogram(premade_audio_data, fs=44100, resampling_factor=7, ymax=800)

In [ ]:
# get_sound_pane(
#     sound_format(premade_audio_data),
#     fs=44100
# )

### Wavelet analysis from SwarmX:TFA

In [ ]:
def transform_using_wavelets(df, t0, t1, cutoff=20/1000, minScale=10, maxScale=100, dj=0.1):
    tfa_params = {'time_lims': [t0, t1]}
    tfa = tfalib_temp.TempTFAdata(df.index, df["F"].values.reshape(len(df["F"].values), 1), params=tfa_params)
    cadence = tfalib_temp.Cadence({'Sampling_Rate': 86400, 'Interp': False})
    cleaning = tfalib_temp.Cleaning({'Window_Size': 50, 'Method': 'iqr', 'Multiplier': 6})
    filtering = tfalib_temp.Filtering({'Sampling_Rate': 1, 'Cutoff': cutoff, })
    wavelet = tfalib_temp.Wavelet(
        {'Time_Step': 1, 'Min_Scale': minScale, 
         'Max_Scale': maxScale, 'dj': dj}
    )
    cadence.apply(tfa)
    cleaning.apply(tfa)
    filtering.apply(tfa)
    wavelet.apply(tfa)
    return tfa

# tfa = transform_using_wavelets(df, t0, t1)

In [ ]:
# tfa.image()

In [ ]:
# x_tfa = tfa.wave_index()

In [ ]:
# audio_data = _apply_audio_pipeline(x_tfa)
# plot_spectrogram(audio_data, fs=44100, resampling_factor=7, ymax=800)

In [ ]:
# get_sound_pane(
#     sound_format(audio_data),
#     fs=44100
# )

### Extend the audio pipeline to handle the dataframe and transformations too...

In [ ]:
def apply_audio_pipeline_to_df(df, t0, t1, target_length=10, shift_octaves=1, transform=False, transform_kwargs=dict()):
    if transform:
        tfa = transform_using_wavelets(df, t0, t1, **transform_kwargs)
        x = tfa.wave_index()
    else:
        x = df["F"].values
        tfa = None
    x = _apply_audio_pipeline(x, target_length=target_length, shift_octaves=shift_octaves)
    return x, tfa

In [ ]:
# x, tfa = apply_audio_pipeline_to_df(df, t0, t1, target_length=10, shift_octaves=1, transform=False)
# get_sound_pane(
#     sound_format(x),
#     fs=44100
# )

In [ ]:
# x, tfa = apply_audio_pipeline_to_df(
#     df, t0, t1, target_length=10, shift_octaves=1, transform=True,
#     transform_kwargs=dict(cutoff=20/1000, minScale=10, maxScale=100, dj=0.1)
# )
# get_sound_pane(
#     sound_format(x),
#     fs=44100
# )

## Panel dashboard

In [ ]:
class SoundDashboard:
    def __init__(self, df=df, t0=t0, t1=t1):
        """Initialise with the data from above, and create Panel objects"""
        self.default_t0t1 = t0, t1
        self.t0 = t0
        self.t1 = t1
        # self.df = fetch_data(t0, t1)
        self.df = df  # use the pre-fetched data from above for speed
        # self.audio_data = apply_audio_pipeline(self.df["F"].values)
        self.audio_data = premade_audio_data  # use the premade data from above for speed
        spectrogram_in = plot_spectrogram(self.df["F"].values)
        spectrogram_out = plot_spectrogram(self.audio_data, fs=44100, resampling_factor=7, ymax=800)
        wavfilename = self.write_file_for_download()
        self.widgets = {
            "time_range": pn.widgets.DatetimeRangeInput(
                start=dt.datetime(2013, 1, 1, 0, 0, 0), end=dt.datetime(2023, 1, 1, 0, 0, 0),
                value=(t0, t1)
            ),
            "button1": pn.widgets.Button(
                name="Fetch data", button_type="primary"
            ),
            "loading1": pn.indicators.Progress(active=False, sizing_mode="stretch_width"),
            "wavelet_transform_checkbox": pn.widgets.Checkbox(name="Apply wavelet transform:"),
            "target_length": pn.widgets.IntSlider(
                name="Output length (seconds)",
                start=1, end=60, step=1, value=10
            ),
            "shift_octaves": pn.widgets.IntSlider(
                name="Shift by number of octaves",
                start=0, end=6, value=1
            ),
            "button2": pn.widgets.Button(
                name="Regenerate sound ➡️", button_type="primary"
            ),
            "loading2": pn.indicators.Progress(active=False, sizing_mode="stretch_width"),
            "file_download": pn.widgets.FileDownload(file=wavfilename)
        }
        self.widgets_wavelet_transform = {
            "cutoff": pn.widgets.EditableFloatSlider(name="Cutoff frequency", start=0.001, end=0.1, value=0.020, step=0.001),
            "minScale": pn.widgets.EditableFloatSlider(name="Wavelet smallest scale", start=1, end=100, value=10, step=10),
            "maxScale": pn.widgets.EditableFloatSlider(name="Wavelet largest scale", start=100, end=1000, value=100, step=10),
            "dj": pn.widgets.EditableFloatSlider(name="Step size for scales for wavelet transform", start=0.05, end=0.15, value=0.1, step=0.01),
        }
        self.panes = {
            "audio": get_sound_pane(sound_format(self.audio_data)),
            "spectrogram_in": pn.pane.Matplotlib(spectrogram_in),
            "spectrogram_out": pn.pane.Matplotlib(spectrogram_out),
            "wavelets_spectrogram": pn.pane.Matplotlib()
        }
        self.widgets["button1"].on_click(self.update_data)
        self.widgets["button2"].on_click(self.update_audio)

    def update_data(self, event):
        """Fetch the data from VirES and reset the dashboard"""
        t0, t1 = self.widgets["time_range"].value
        self.t0, self.t1 = t0, t1
        if (t1 - t0) > dt.timedelta(days=7):
            self.widgets["button1"].name = "Time > 7 days not allowed !"
            self.widgets["time_range"].value = self.default_t0t1
            sleep(3)
            self.widgets["button1"].name = "Fetch data"
            return None
        self.widgets["button1"].name = "Busy..."
        self.widgets["loading1"].active = True
        self.df = fetch_data_padded(t0, t1)
        try:
            spectrogram_in = plot_spectrogram(self.df["F"].values)
        except KeyError:
            self.widgets["button1"].name = "Missing data for this time..."
            self.widgets["time_range"].value = self.default_t0t1
            sleep(3)
            self.widgets["button1"].name = "Fetch data"
            self.widgets["loading1"].active = False
            return None
        self.panes["spectrogram_in"].object = spectrogram_in
        self.widgets["button1"].name = "Fetch data"
        self.widgets["loading1"].active = False
        self.update_audio(None)

    def update_audio(self, event):
        try:
            self._update_audio()
        except Exception:
            self.widgets["button2"].name = "Oops! Something went wrong"
            self.widgets["loading2"].active = False

    def _update_audio(self):
        """Update the output spectrogram and audio"""
        self.panes["audio"].paused = True
        self.widgets["button2"].name = "Busy..."
        self.panes["wavelets_spectrogram"].object = None
        # Change contents of audio and spectrogram
        self.widgets["loading2"].active = True
        # Extract kwargs to pass to the wavelet transform
        wavelet_transform_kwargs = {k: v.value for k, v in self.widgets_wavelet_transform.items()}
        # Apply the pipeline
        x, tfa = apply_audio_pipeline_to_df(
            self.df,
            self.t0,
            self.t1,
            target_length=self.widgets["target_length"].value,
            shift_octaves=self.widgets["shift_octaves"].value,
            transform=self.widgets["wavelet_transform_checkbox"].value,
            transform_kwargs=wavelet_transform_kwargs
        )
        # Insert the wavelet spectrogram
        if tfa:
            self.panes["wavelets_spectrogram"].object = tfa.image()
        # Update the audio
        self.audio_data = x
        self.panes["audio"].object = sound_format(x)
        spectrogram_out = plot_spectrogram(x, fs=44100, resampling_factor=7, ymax=800)
        self.panes["spectrogram_out"].object = spectrogram_out
        self.write_file_for_download()
        # Reset button & loading widget
        self.widgets["button2"].name = "Regenerate sound ➡️"
        self.widgets["loading2"].active = False

    def write_file_for_download(self, filename="sonification.wav"):
        wavfile.write(filename, 44100, sound_format(self.audio_data))
        return filename

    def display(self):
        """GridSpec-based layout of all the widgets and panes"""
        gspec = pn.GridSpec(sizing_mode="stretch_both", max_height=800)
        gspec[:, 0] = pn.Column(
            pn.pane.Markdown("## 1. Input data"),
            pn.pane.Markdown("`SW_OPER_MAGA_LR_1B: F`"),
            self.widgets["time_range"],
            self.widgets["button1"],
            self.widgets["loading1"],
            pn.pane.Markdown("### Input spectrogram"),
            self.panes["spectrogram_in"]
        )
        gspec[:, 1] = pn.Column(
            pn.pane.Markdown("## 2. Transformations"),
            pn.layout.spacer.VSpacer(max_height=10),
            pn.Column(
                self.widgets["wavelet_transform_checkbox"],
                *self.widgets_wavelet_transform.values(),
                pn.pane.Markdown("*To apply these, tick the checkbox above*"),
                background="WhiteSmoke",
                align="center"
            ),
            pn.layout.spacer.VSpacer(max_height=10),
            pn.Column(
                self.widgets["target_length"],
                self.widgets["shift_octaves"],
                self.widgets["button2"],
                background="WhiteSmoke",
                align="center"
            ),
            pn.layout.spacer.VSpacer(max_height=10),
            self.panes["wavelets_spectrogram"],
            background="Snow"
        )
        gspec[:, 2] = pn.Column(
            pn.pane.Markdown("## 3. Output spectrogram & audio"),
            self.widgets["loading2"],
            self.panes["spectrogram_out"],
            self.panes["audio"],
            self.widgets["file_download"]
        )
        return gspec


SoundDashboard().display().servable("Sounds from Swarm")